<a href="https://colab.research.google.com/github/tsparaskevas/DataJournalism/blob/main/Processing%20and%20analyzing%20tweets/processing_%26_analyzing_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Processing & Analyzing tweets**

**ΣΗΜΕΙΩΣΗ:** Αυτό τo notebook στοχεύει στην επεξήγηση της διαδικασίας και στην επανάληψη. Εσείς στην εργασία σας θα χρησιμοποιήσετε από εδώ ό,τι θέλετε και αν θέλετε.

Καλή επιτυχία!!!

###**Εγκατάσταση της Spacy (για επεξεργασία κειμένων στα ελληνικά)**

1. Αρχικά κάνουμε εγκατάσταση (σβήνουμε το ```#``` και τρέχουμε το cell)
2. Μόλις εγκατασταθεί ξαναβάζουμε το ```#```
3. Κάνουμε Runtime > Restart runtime



In [ ]:
#!python -m spacy download el_core_news_sm

###**Εισαγωγή βιβλιοθηκών**

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


###**Εισαγωγή και προετοιμασία των δεδομένων (tweets sets)**

Αλλάζουμε τις ρυθμίσεις στο display των pandas για να μπορούμε να δούμε περισσότερα μέσα στα cells

In [2]:
# Μεγιστοποίηση του πλάτους της στήλης για εμφάνιση ολόκληρου του κειμένου
pd.set_option('display.max_colwidth', None)

####**Πρώτο dataset** (δώστε το όνομα του δικού σας πρώτου dataset)

Στον παρακάτω κώδικα δώστε το όνομα που θέλετε να έχει το full_tweets_df και κάντε τις απαραίτητες αλλαγές όπου χρειάζεται.

In [ ]:
# Import & View data

# hint: όπως μας έδειξε ο Χάρης, μπορούμε να βρούμε και να κάνουμε copy paste το path του επιθυμητού tsv από το εικονίδιο του φακέλου στο κάθετο μενού αριστερά
onoma_a_full_tweets_df = pd.read_csv("το/πλήρες/path/του/tsv/arxeiou", sep='\t') 
# View data
print(onoma_a_full_tweets_df.shape)
print(f"dtype of 'created_at' is: {onoma_a_full_tweets_df.created_at.dtypes}. To θέλουμε σε datetime64[ns]") #
onoma_a_full_tweets_df.head(1)

In [ ]:
# Convert date to datetime to a new column (με την εντολή to_datetime) 
onoma_a_full_tweets_df['datetime'] = pd.to_datetime(onoma_a_full_tweets_df['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')

In [ ]:
# Make a dataframe with selected columns only
onoma_a_tweets_df = onoma_a_full_tweets_df[['datetime', "full_text"]]

# Extract year, month and day from daytime
onoma_a_tweets_df['year'] = pd.DatetimeIndex(onoma_a_tweets_df['datetime']).year
onoma_a_tweets_df['month'] = pd.DatetimeIndex(onoma_a_tweets_df['datetime']).month
onoma_a_tweets_df['day'] = pd.DatetimeIndex(onoma_a_tweets_df['datetime']).day

# View dataframe info
print(onoma_a_tweets_df.info())
onoma_a_tweets_df.head(1)

Φτιάξτε ένα γράφημα για το πλήθος των tweets ανά έτος

In [ ]:
# Make a barplot of tweets per year
onoma_a_tweets_df['year'].value_counts().plot(kind='bar')

**Καθάρισμα του κειμένου**

In [ ]:
# Remove any urls in tweets' full_text (replace with space)
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['full_text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

Μπορείτε να κάνετε και περισσότερο καθάρισμα στο κείμενο από σκουπίδια, με τη χρήση regex (regular expressions = κανονικές εκφράσεις) που θα μάθουμε σε επόμενο μάθημα.

In [ ]:
# Replace unwanted characters with spaces or nothing
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['clean_text'].str.replace(r'[\n]*\s*https?:\/\/.*[\r\n]*'," ").str.replace(r'\n*',"").str.replace(r'[▪️🟢⚪👉]*',"").str.replace(r"\w\/\w", "")
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['clean_text'].str.replace('amp;', '').str.replace(r'[\!\-\:\,\.«»\"“”\[–()&]'," ").str.replace('&gt;', ' ').str.replace(r"\s\s\s?", ' ')
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['clean_text'].str.replace(r"\w*…", "")

Αν θέλετε μπορείτε να πετάξετε και τα RTs, τα hashtags και τα @names για να έχετε μόνο το κείμενο

In [ ]:
# Aφαίρεση των #hashtags, των @names και των RT
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['clean_text'].str.replace(r'RT\s', '').str.replace(r'@[\w\_]*\s', '').str.replace(r"#[\w\W\d]*","")#.str.replace(r'http[s\s/\\.]*', '')

Ελέγξτε πώς πήγε το καθάρισμα συγκρίνοντας το full_text με το clean_text

In [ ]:
# Check cleaning results
onoma_a_tweets_df.loc[15:20][['full_text', 'clean_text']] # αλλάζοντας τα νούμερα στο loc, επισκοπω διαφορετικά rows 

In [ ]:
# Μετατροπή του κειμένου σε lowercase
onoma_a_tweets_df['clean_text'] = onoma_a_tweets_df['clean_text'].str.lower()

# View dataframe
onoma_a_tweets_df.head(1)

Μετρήστε τις λέξεις του κάθε tweet (του κάθε row)

In [ ]:
onoma_a_tweets_df['clean_text'].str.split().str.len() 

Προσθέστε στο τέλος του προηγούμενου κώδικα ό,τι χρειάζεται για να δείτε το μ.ο. των λέξεων στα tweets

Μπορείτε να σκεφτείτε άλλα ερωτήματα και να πάρετε τις απαντήσεις;

####**Δεύτερο dataset** (δώστε το όνομα του δικού σας δεύτερου dataset)

Επαναλάβετε τα παραπάνω για το δεύτερο dataset των tweets

###**Wordclouds**

####**Εισαγωγή βιβλιοθηκών**

In [ ]:
from wordcloud import WordCloud, STOPWORDS

####**Πρώτο dataset** (δώστε το όνομα του δικού σας πρώτου dataset)

Δημιουργία ενός text που θα περιέχει τα κείμενα από όλα τα tweets του dataset (δεν έχει νόημα να κάνουμε wordcloud από ένα - ένα τα tweets. Θέλουμε να δούμε τι γίνεται σε μεγάλα κείμενα π.χ. σε ολόκληρα βιβλία, σε πολιτικούς λόγους, σε μεγάλα σύνολα δημοσιευμάτων κλπ., άρα θέλουμε να δούμε τι γίνεται στο λόγο που χρησιμοποιεί ένας twitter user σε όλα τα tweets του που διαθέτουμε)

In [ ]:
# Concatenate tweets' texts
text_a = onoma_a_tweets_df['clean_text'].str.cat(sep = ' ')#.replace("amp", ' ')
# View text
text_a

**Σύντομη επανάληψη**

Θέλουμε να φτιάξουμε ένα συννεφόλεξο, δηλαδή ένα γραφικό στο οποίο οι λέξεις που περιέχονται σε ένα κείμενο εμφανίζονται με διαφορετικό μέγεθος γραμμάτων ανάλογα με τη συχνότητα εμφάνισής τους.

Οι συχνότερες όμως λέξεις σε ένα κείμενο είναι τα άρθρα, οι σύνδεσμοι κλπ. Για να μην εμφανίζονται αυτές οι λέξεις, χρησιμοποιούμε ένα φίλτρο για να μην τις μετράει το wordcloud. Αυτό το φίλτρο στην ουσία είναι ένα λεξικό με ανεπιθύμητες λέξεις που ονομάζεται STOPWORDS.

Όπως θα θυμάστε από το μάθημα, τα STOPWORDS που περιέχει το WordCloud είναι στα αγγλικά, άρα δεν μας είναι χρήσιμο σε ελληνικά κείμενα. Γι' αυτό χρησιμοποιούμε τα ελληνικά stopwords που έχει η spacy.

**WordCloud**

In [ ]:
# Import spacy
import spacy
# Load small version of spacy's dictionary that contains greek words used in news
nlp = spacy.load('el_core_news_sm')

In [ ]:
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words, # stopwords from 'el_core_news_sm' dictionary
    width = 2000,
    height = 1000,
    background_color = 'black'
    ).generate(text_a) # εδώ δηλώνουμε από ποιο κείμενο θα φτιάξει το wordcloud
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

Όπως βλέπουμε, ίδιες λέξεις που βρίσκονται σε διαφορετικές πτώσεις/αριθμούς/χρόνους/πρόσωπα, εμφανίζονται σαν διαφορετικές λέξεις. Για να λύσουμε αυτό το πρόβλημα, χρησιμοποιούμε τo lemmatization της spacy (που έχουμε φορτώσει σαν nlp)...

In [ ]:
doc_a = nlp(text_a) # περνάμε το text από την nlp
lemma_text_a = ' '.join(token.lemma_ for token in doc_a) # εφαρμόζουμε lemmatization σε κάθε λέξη

...και ξαναφτιάχνουμε το wordcloud

In [ ]:
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words, 
    width = 2000,
    height = 1000,
    background_color = 'black'
    ).generate(lemma_text_a) # βάζουμε το lemmatized text
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

####**Δεύτερο dataset** (δώστε το όνομα του δικού σας δεύτερου dataset)

Επαναλάβετε τα παραπάνω για το δεύτερο dataset των tweets

###**Challenge επανάληψης:**

Μπορείτε να συνενώσετε τα δύο dataframes (merge) ώστε να πάρετε ωραία συγκριτικά γραφήματα για τα στατιστικά τους στοιχεία;